In [ ]:
import flopy
import pyemu
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import herebedragons as hbd
import shutil

# Intro

This notebook runs a prior monte carlo using the pest setup generated in the `setup_pstfrom` notebook. We will run the prior parameter ensemble once and stop. We will take a look at some of the results.

### Warning
This notebook can take some time to run. That is the cost of a >5min model run time...imagine if your model takes longer than that (#suffering). For context, this takes about 60min on a MacBook Pro. Expect longer on Windows. We recommend setting this to run over night or when you go out for lunch, before proceeding to the next notebooks. 

In [ ]:
# specify the temporary working folder
t_d = os.path.join('pst_template_pmc')
# get the previously generated PEST dataset
org_t_d = os.path.join('pst_template')
if not os.path.exists(org_t_d):
    raise Exception()
if os.path.exists(t_d):
    shutil.rmtree(t_d)
shutil.copytree(org_t_d,t_d)

# Prior Monte Carlo

Load the pest control file

In [ ]:
pst = pyemu.Pst(os.path.join(t_d,"pest.pst"))

Lets run 100 realizations (recall we actualy generated 1000)

In [ ]:
pst.pestpp_options["ies_num_reals"] = 100
pst.pestpp_options["save_binary"] = True

Set noptmax=-1 to just run the ensemble once. And lets store results in binary format for speed and disk savings...

In [ ]:
pst.control_data.noptmax = -1
pst.write(os.path.join(t_d,"pest.pst"),version=2)


### Warning: set num_workers according to your reserouces

In [ ]:
num_workers=10
m_d = "master_prior_mc"

In [ ]:

pyemu.os_utils.start_workers(t_d, # the folder which contains the "template" PEST dataset
                            'pestpp-ies', #the PEST software version we want to run
                            'pest.pst', # the control file to use with PEST
                            num_workers=num_workers, #how many agents to deploy
                            worker_root='.', #where to deploy the agent directories; relative to where python is running
                            master_dir=m_d, #the manager directory
                            )

## Load results

In [ ]:
pst = pyemu.Pst(os.path.join(m_d,"pest.pst"))

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_ws=m_d, load_only=[],verbosity_level=0)
gwf = sim.get_model('gwf')

In [ ]:
obs = pst.observation_data
obs.oname.unique()

In [ ]:
oe = pst.ies.obsen.copy()
oe.head()

## Make some figures

Lets look at some of the simualted max temperature fields.

In [ ]:
i = 'base'#oe.index.values[1]
oname = "temp"

for i in oe.index.values[-5:]:
    fig,ax = plt.subplots(1,1,figsize=(6,6))


    ax.set_aspect("equal")
    pm = flopy.plot.PlotMapView(model=gwf, ax=ax)
    #pm.plot_grid(alpha=0.1,lw=0.1)

    _obs = obs.loc[obs.oname==oname].copy()
    _obs["i"] = _obs["i"].astype(int)
    _obs.sort_values("i", inplace=True)
    obsnmes = _obs.obsnme.tolist()
    diverg = max(oe.loc[:,obsnmes].values.max(), abs(oe.loc[:,obsnmes].values.min()))
    vmax = np.ceil(diverg)
    vmin = 16 
    
    arr = oe.loc[i,obsnmes].values
    #arr = arr.reshape(61,gwf.modelgrid.ncpl).max(axis=0)
    #f oname=='k':
    #    arr = np.log10(arr)

    pa = pm.plot_array(arr, cmap="Reds", vmin=vmin,vmax=vmax)
    plt.colorbar(pa, ax=ax, shrink=0.5)

    ax.set_title(oname)
    ax.set_xticks([])
    ax.set_yticks([])

    fig.tight_layout();
    plt.show()
    plt.close();